In [ ]:
def cnn_model_f(x,y,k1,k2,dk1,dk2,db1,db2,b1,b2,decimal_b,float_b):
    r1 = np.array([])
    r2 = np.array([])
    l1_out1 = np.empty((0,26),float)
    l1_out2 = np.empty((0,26),float)

    # layer 1 convolution
    for i in range(26):
        for j in range(26):
            convo_r1 = 0
            convo_r2 = 0
            for m in range(3):
                for n in range(3):
                    tmp1 = x[i+m,j+n]*k1[m,n]
                    tmp2 = x[i+m,j+n]*k2[m,n]
                    convo_r1 += tmp1-tmp1%(2**float_b)
                    convo_r2 += tmp2-tmp2%(2**float_b)
            convo_r1 += b1
            convo_r2 += b2
            # relu
            convo_r1 = max(convo_r1,0)
            convo_r2 = max(convo_r2,0)
            r1 = np.append(r1,convo_r1)
            r2 = np.append(r2,convo_r2)
        l1_out1 = np.append(l1_out1,np.expand_dims(r1, axis=0),axis=0)
        l1_out2 = np.append(l1_out2,np.expand_dims(r2, axis=0),axis=0)
        r1 = np.array([])
        r2 = np.array([])
        
    
    # layer 2 max pooling

    maxp1 = -1
    maxp2 = -1
    l2_out1 = np.empty((0,13),float)
    l2_out2 = np.empty((0,13),float)
    for i in range(0,25,2):
        for j in range(0,25,2):
            for m in range(2):
                for n in range(2):
                    maxp1 = max(maxp1,l1_out1[i+m,j+n]) 
                    maxp2 = max(maxp2,l1_out2[i+m,j+n])
            r1 = np.append(r1,maxp1)
            r2 = np.append(r2,maxp2)
            maxp1 = -1
            maxp2 = -1
        l2_out1 = np.append(l2_out1,np.expand_dims(r1, axis=0),axis=0)
        l2_out2 = np.append(l2_out2,np.expand_dims(r2, axis=0),axis=0)  
        r1 = np.array([])
        r2 = np.array([])


    # layer 3 flatten

    l3_out = np.array([])

    for i in range(13):
        for j in range(13):
            l3_out = np.append(l3_out,l2_out1[i,j])
            l3_out = np.append(l3_out,l2_out2[i,j])
 

    # layer 4 dense

    l4_out = np.dot(l3_out,dk1)
    for i in range(128):
        l4_out[i] = max(l4_out[i]+db1[i],0)

    
    # layer 5 dense with softmax

    l5_out = np.dot(l4_out,dk2)
    for i in range(10):
        l5_out[i] = l5_out[i]+db2[i]

 
    ans = np.exp(l5_out)/sum(np.exp(l5_out))
    
    #if decimal_b==0:
    #print(ans)
        
    v = 0
    ma = -1
    for i in range(10):
        if ans[i]>ma:
            ma = ans[i]
            v = i
    
    if y[v]>0.6:
        return True
    else: 
        return False